#Disruptive Architectures - Challenge Sprint 2


## Integrantes do grupo

  | Nome                         | RM     |
  |------------------------------|--------|
  | Natan Junior Rodrigues Lopes | 552626 |
  | Pedro Lucca Medeiros Miranda | 553873 |
  | Pedro Moreira de Jesus       | 553912 |

## Contextualizando

Nesta segunda Sprint da matéria de Disruptive Architectures, nós desempenhamos o trabalho de análise de dados, necessário para a de uma versão inicial do nosso modelo de Inteligência Artificial.

Após um estudo dos principais fatores comuns envolvidos na materialização de risco para empresas na área da saúde odontológica, nós montamos a primeira versão de nosso Dataset utilizando uma biblioteca de Mocking chamada *Faker*.

## Instalando o Faker e importando as dependências

In [2]:
!pip install faker
import pandas as pd
import numpy as np
from faker import Faker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Criando o dataset com dados falsos de teste

Para criar nosso dataset, fizemos uma análise de dados que possuem relação com o acontecimento do sinistro odontológico. Entendendo a relação desses fatores e seu impacto no _target_ sinistro, nós conseguimos montar nossa tabela utilizando o Pandas, Numpy e o Faker.

In [3]:
fake = Faker()
num_records = 10000

data = {
    'idade': np.random.randint(18, 80, size=num_records),
    'genero': np.random.choice(['M', 'F'], size=num_records),
    'localizacao': [fake.city() for _ in range(num_records)],
    'frequencia_consultas': np.random.poisson(2, num_records),
    'aderencia_tratamento': np.random.uniform(0, 1, num_records),
    'historico_caries': np.random.binomial(1, 0.3, size=num_records),
    'doenca_periodontal': np.random.binomial(1, 0.2, size=num_records),
    'numero_implantes': np.random.randint(0, 5, size=num_records),
    'tratamentos_complexos_previos': np.random.binomial(1, 0.3, size=num_records),
    'fumante': np.random.binomial(1, 0.2, size=num_records),
    'alcoolismo': np.random.binomial(1, 0.1, size=num_records),
    'escovacao_diaria': np.random.choice([1, 2, 3], size=num_records, p=[0.2, 0.6, 0.2]),
    'uso_fio_dental': np.random.binomial(1, 0.5, size=num_records),
    'doencas_sistemicas': np.random.choice(['Diabetes', 'Nenhuma', 'Cardíaca', 'Outras'], size=num_records, p=[0.1, 0.7, 0.1, 0.1]),
    'medicamentos_uso_continuo': np.random.poisson(1, num_records),
    'numero_sinistros_previos': np.random.poisson(1, num_records),
    'valor_medio_sinistros': np.round(np.random.uniform(200, 2000, num_records), 2),
    'tipo_plano': np.random.choice(['basico', 'intermediario', 'avancado'], size=num_records, p=[0.5, 0.3, 0.2]),
}

df = pd.DataFrame(data)

df['probabilidade_sinistro'] = 0.1

# Aqui, nós combinamos nossos dados para gerar a probabilidade de sinistro, nossa coluna target.
df['probabilidade_sinistro'] += df['idade'].apply(lambda x: 0.05 if x > 60 else 0)
df['probabilidade_sinistro'] += df['doenca_periodontal'] * 0.1
df['probabilidade_sinistro'] += df['historico_caries'] * 0.05
df['probabilidade_sinistro'] += df['fumante'] * 0.1
df['probabilidade_sinistro'] += df['alcoolismo'] * 0.05
df['probabilidade_sinistro'] += df['tratamentos_complexos_previos'] * 0.08
df['probabilidade_sinistro'] += df['numero_sinistros_previos'] * 0.05
df['probabilidade_sinistro'] += df['doencas_sistemicas'].apply(lambda x: 0.1 if x != 'Nenhuma' else 0)

# Já nesta etapa, reduzimos a probabilidade de sinistro com base em fatores
# como a prevenção de problemas com a saúde bucal e o tratamento de doenças
df['probabilidade_sinistro'] -= df['frequencia_consultas'] * 0.02
df['probabilidade_sinistro'] -= df['aderencia_tratamento'] * 0.1

df['probabilidade_sinistro'] = df['probabilidade_sinistro'].clip(0, 1)

## Dataset criado

Por fim, esta foi a estrutura do Dataset:

In [4]:
df

,idade,genero,localizacao,frequencia_consultas,aderencia_tratamento,historico_caries,doenca_periodontal,numero_implantes,tratamentos_complexos_previos,fumante,alcoolismo,escovacao_diaria,uso_fio_dental,doencas_sistemicas,medicamentos_uso_continuo,numero_sinistros_previos,valor_medio_sinistros,tipo_plano,probabilidade_sinistro
0,64,F,Jennifershire,6,0.307256,0,0,2,1,0,1,2,1,Diabetes,0,0,1124.91,intermediario,0.229274
1,63,F,New Christopherfurt,0,0.905568,0,1,1,1,0,0,2,1,Nenhuma,2,1,839.48,basico,0.289443
2,36,M,North George,0,0.333692,1,0,2,0,0,1,1,0,Nenhuma,1,2,1730.56,avancado,0.266631
3,60,F,Jonesshire,0,0.617681,1,1,1,0,0,0,3,1,Nenhuma,3,0,1302.93,basico,0.188232
4,44,F,Toddhaven,1,0.621212,0,0,3,0,1,0,1,0,Nenhuma,1,1,1012.15,intermediario,0.167879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,45,M,Port Sarah,1,0.811788,0,0,4,0,0,0,3,0,Cardíaca,1,0,1792.94,basico,0.098821
9996,41,F,Joshuafort,2,0.315257,0,0,4,0,0,0,1,1,Nenhuma,0,2,319.71,intermediario,0.128474
9997,42,M,Lake Olivia,2,0.586943,0,0,3,0,0,0,2,1,Nenhuma,4,1,1729.91,intermediario,0.051306
9998,63,F,New Robertside,2,0.287289,0,1,3,1,0,0,2,0,Nenhuma,1,1,1804.29,intermediario,0.311271


##Treinamento do modelo

Em seguida, fizemos o tratamento e organização dos dados para iniciar o treinamento do modelo


Primeiro, utilizamos o Label Encoder para transformar nossos dados textuais e categóricos em numéricos

In [5]:
label_encoder = LabelEncoder()

df['localizacao'] = label_encoder.fit_transform(df['localizacao'])
df['doencas_sistemicas'] = label_encoder.fit_transform(df['doencas_sistemicas'])
df['tipo_plano'] = label_encoder.fit_transform(df['tipo_plano'])
df['genero'] = label_encoder.fit_transform(df['genero'])

X = df.drop('probabilidade_sinistro', axis=1)
y = df['probabilidade_sinistro']
X = pd.get_dummies(X, drop_first=True)

### Instanciando o GBR

Em seguida, dividimos o Dataset em treino e teste, com 20% de dados para teste e
por fim, nós instanciamos um modelo de regressão chamado Gradient Boosting Regressor, uma vez que ao trabalhar com valores percentuais, um modelo de regressão é o mais adequado. Além disso, devido a estrutura e baixa complexidade de nosso Dataset, Redes Neurais não seriam tão eficientes.

In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingRegressor()
model.fit(X_train, y_train)


GradientBoostingRegressor()

E então, testamos o modelo:

In [7]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'R²: {r2:.4f}')

MSE: 0.0002
R²: 0.9872


## Teste Final
Para garantir a qualidade sua qualidade, geramos dois dados de teste e realizamos o predict com o nosso modelo

In [9]:
new_data = pd.DataFrame({
    'idade': [30, 65],
    'genero': ['M', 'F'],
    'localizacao': ['Cidade A', 'Cidade B'],
    'frequencia_consultas': [3, 1],
    'aderencia_tratamento': [0.8, 0.5],
    'historico_caries': [0, 1],
    'doenca_periodontal': [1, 0],
    'numero_implantes': [0, 2],
    'tratamentos_complexos_previos': [0, 1],
    'fumante': [0, 1],
    'alcoolismo': [0, 0],
    'escovacao_diaria': [2, 1],
    'uso_fio_dental': [1, 0],
    'doencas_sistemicas': ['Nenhuma', 'Diabetes'],
    'medicamentos_uso_continuo': [0, 2],
    'numero_sinistros_previos': [0, 1],
    'valor_medio_sinistros': [500, 1500],
    'tipo_plano': ['intermediario', 'avancado'],
})

label_encoder = LabelEncoder()
new_data['localizacao'] = label_encoder.fit_transform(new_data['localizacao'])
new_data['doencas_sistemicas'] = label_encoder.fit_transform(new_data['doencas_sistemicas'])
new_data['tipo_plano'] = label_encoder.fit_transform(new_data['tipo_plano'])
new_data['genero'] = label_encoder.fit_transform(new_data['genero'])

new_data = pd.get_dummies(new_data, drop_first=True)

predictions = model.predict(new_data)

print(predictions)
limiar = 0.4
classificacoes = ["Alto Risco" if prob > limiar else "Baixo Risco" for prob in predictions]
print(classificacoes)

[0.16702486 0.44496488]
['Baixo Risco', 'Alto Risco']
